In [62]:
import torch
from torchvision import datasets, transforms,models
import torch.nn as nn
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
import numpy as np

In [63]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))    
])

In [64]:
train_data = datasets.CIFAR10(root="./data",train=True,download=True,transform=transform)
test_data = datasets.CIFAR10(root=".'data",train=False,download=True,transform=transform)

In [65]:
train_loader = DataLoader(train_data,batch_size=16,shuffle=True)
test_loader = DataLoader(test_data,batch_size=len(test_data),shuffle=False)

In [66]:
net = models.resnet18(pretrained=True)
net.parameters

<bound method Module.parameters of ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)


In [67]:

for param in net.layer1.parameters():
    param.requires_grad = False
for param in net.layer2.parameters():
    param.requires_grad = False

net.fc = nn.Linear(net.fc.in_features,10)
net.fc.requires_grad = True

In [68]:
leaning_rate = 0.0005
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(),lr=leaning_rate)

In [69]:
def train(net,n_epoch=2):
    for epoch in tqdm(range(n_epoch)):
        batchiter = iter(train_loader)

        running_loss = 0.0
        all_preds = []  # Инициализируем перед циклом
        all_labels = []
        
        for i,batch in enumerate(tqdm(batchiter)):
            x_batch,y_batch = batch
            optimizer.zero_grad()
            y_pred=net(x_batch)
            loss = loss_fn(y_pred,y_batch)
            loss.backward()
            optimizer.step()
            running_loss+=loss.item()
            
            _, preds = torch.max(y_pred, dim=1)  # Получаем классы
            all_preds.extend(preds.cpu().numpy())  # Переводим в numpy
            all_labels.extend(y_batch.cpu().numpy())

            if i % 500 == 499:  # Каждые 500 итераций
                avg_loss = running_loss / 500
                accuracy = accuracy_score(all_labels, all_preds) * 100
                print(f'[{epoch + 1}, {i + 1}] loss: {avg_loss:.3f} | accuracy: {accuracy:.2f}%')
                
                running_loss = 0.0
                all_preds = []
                all_labels = []  # Сброс списков
    print("обучение завершено")
    return net


In [70]:
train(net,10)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/3125 [00:00<?, ?it/s]

[1, 500] loss: 1.775 | accuracy: 37.60%
[1, 1000] loss: 1.469 | accuracy: 48.56%
[1, 1500] loss: 1.372 | accuracy: 52.55%
[1, 2000] loss: 1.318 | accuracy: 54.43%
[1, 2500] loss: 1.290 | accuracy: 55.16%
[1, 3000] loss: 1.233 | accuracy: 56.55%


  0%|          | 0/3125 [00:00<?, ?it/s]

[2, 500] loss: 1.198 | accuracy: 58.59%
[2, 1000] loss: 1.175 | accuracy: 58.56%
[2, 1500] loss: 1.153 | accuracy: 59.89%
[2, 2000] loss: 1.147 | accuracy: 60.36%
[2, 2500] loss: 1.117 | accuracy: 60.88%
[2, 3000] loss: 1.134 | accuracy: 61.20%


...

  0%|          | 0/3125 [00:00<?, ?it/s]

[9, 500] loss: 0.813 | accuracy: 72.05%
[9, 1000] loss: 0.834 | accuracy: 71.44%
[9, 1500] loss: 0.839 | accuracy: 71.38%
[9, 2000] loss: 0.820 | accuracy: 71.66%
[9, 2500] loss: 0.834 | accuracy: 71.25%
[9, 3000] loss: 0.820 | accuracy: 71.91%


  0%|          | 0/3125 [00:00<?, ?it/s]

[10, 500] loss: 0.797 | accuracy: 72.66%
[10, 1000] loss: 0.802 | accuracy: 71.79%
[10, 1500] loss: 0.840 | accuracy: 71.11%
[10, 2000] loss: 0.816 | accuracy: 71.58%
[10, 2500] loss: 0.828 | accuracy: 71.24%
[10, 3000] loss: 0.809 | accuracy: 71.67%
обучение завершено


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [71]:
test_dataiter = iter(test_loader)
images,labels = next(test_dataiter)

In [72]:

accuracy_score(labels.numpy(),np.argmax(net.forward(images).detach().numpy(),axis=1))

0.7353